In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = high/low
    RS = (R)**2
    
    return nse3, pbias3, pred_day, labels, mae, rmse, RS, R

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'han_pull'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 15)

In [12]:
run_num = range(len(folder))
#run_num = [0]
#if file_parameters['watershed'] == 'nak':
#    run_num = range(len(folder))
#else:
#    run_num = [0]



real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/han/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['가평_2016.xlsx', '가평_2017.xlsx', '가평_2018.xlsx'], ['서상_2016.xlsx', '서상_2017.xlsx', '서상_2018.xlsx'], ['의암호_2016.xlsx', '의암호_2017.xlsx', '의암호_2018.xlsx']]
Epoch 1/2000
1/1 [==============================] - 0s 194ms/step - gen_loss: 109.3603 - disc_loss: 0.7229 - rmse: 1.1239 - val_loss: 0.9885
Epoch 2/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 86.5350 - disc_loss: 0.5437 - rmse: 0.8968 - val_loss: 0.9031
Epoch 3/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 81.3909 - disc_loss: 0.4503 - rmse: 0.9057 - val_loss: 0.8144
Epoch 4/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 71.4632 - disc_loss: 0.4418 - rmse: 0.8425 - val_loss: 0.8295
Epoch 5/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 58.9887 - disc_loss: 0.3816 - rmse: 0.7646 - val_loss: 0.7357
Epoch 6/2000
1/1 [================

1/1 [==============================] - 0s 20ms/step - gen_loss: 8.1045 - disc_loss: 0.3249 - rmse: 0.3748 - val_loss: 0.3615
Epoch 58/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.9985 - disc_loss: 0.3117 - rmse: 0.3326 - val_loss: 0.3281
Epoch 59/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8807 - disc_loss: 0.2952 - rmse: 0.3209 - val_loss: 0.3460
Epoch 60/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3448 - disc_loss: 0.3257 - rmse: 0.3721 - val_loss: 0.3388
Epoch 61/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.7821 - disc_loss: 0.3391 - rmse: 0.4572 - val_loss: 0.3452
Epoch 62/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.2184 - disc_loss: 0.3038 - rmse: 0.3733 - val_loss: 0.3300
Epoch 63/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.7430 - disc_loss: 0.3023 - rmse: 0.4691 - val_loss: 0.2861
Epoch 64/2000
1/1 [======================

Epoch 116/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4515 - disc_loss: 0.2594 - rmse: 0.3276 - val_loss: 0.3356
Epoch 117/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6748 - disc_loss: 0.2498 - rmse: 0.4398 - val_loss: 0.3223
Epoch 118/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.7694 - disc_loss: 0.2347 - rmse: 0.3639 - val_loss: 0.3355
Epoch 119/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1014 - disc_loss: 0.2452 - rmse: 0.3469 - val_loss: 0.3783
Epoch 120/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6860 - disc_loss: 0.2490 - rmse: 0.2904 - val_loss: 0.3005
Epoch 121/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6224 - disc_loss: 0.2609 - rmse: 0.2666 - val_loss: 0.3222
Epoch 122/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5995 - disc_loss: 0.2612 - rmse: 0.3455 - val_loss: 0.2913
Epoch 123/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7203 - disc_loss: 0.2211 - rmse: 0.4163 - val_loss: 0.5409
Epoch 175/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.0296 - disc_loss: 0.2200 - rmse: 0.2523 - val_loss: 0.2587
Epoch 176/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6795 - disc_loss: 0.2226 - rmse: 0.3031 - val_loss: 0.2860
Epoch 177/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 9.0919 - disc_loss: 0.2443 - rmse: 0.3855 - val_loss: 0.3097
Epoch 178/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9330 - disc_loss: 0.2497 - rmse: 0.2946 - val_loss: 0.2620
Epoch 179/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8428 - disc_loss: 0.2355 - rmse: 0.3590 - val_loss: 0.2896
Epoch 180/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0993 - disc_loss: 0.2186 - rmse: 0.3014 - val_loss: 0.4163
Epoch 181/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9680 - disc_loss: 0.2263 - rmse: 0.3397 - val_loss: 0.3189
Epoch 233/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7697 - disc_loss: 0.2270 - rmse: 0.3237 - val_loss: 0.3240
Epoch 234/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2179 - disc_loss: 0.2163 - rmse: 0.2809 - val_loss: 0.3065
Epoch 235/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 11.0026 - disc_loss: 0.2176 - rmse: 0.3241 - val_loss: 0.2401
Epoch 236/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 4.0313 - disc_loss: 0.2098 - rmse: 0.3119 - val_loss: 0.2691
Epoch 237/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3355 - disc_loss: 0.2362 - rmse: 0.2692 - val_loss: 0.2855
Epoch 238/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 9.4814 - disc_loss: 0.2234 - rmse: 0.2331 - val_loss: 0.2469
Epoch 239/2000
1/1 [==============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 0.7123 - disc_loss: 0.0904 - rmse: 0.1332 - val_loss: 0.0670
Epoch 1787/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.8155 - disc_loss: 0.0941 - rmse: 0.0786 - val_loss: 0.1076
Epoch 1788/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.7037 - disc_loss: 0.0920 - rmse: 0.0518 - val_loss: 0.0717
Epoch 1789/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7553 - disc_loss: 0.0859 - rmse: 0.1005 - val_loss: 0.0734
Epoch 1790/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6925 - disc_loss: 0.0904 - rmse: 0.0756 - val_loss: 0.0480
Epoch 1791/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.6739 - disc_loss: 0.0876 - rmse: 0.0550 - val_loss: 0.0680
Epoch 1792/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.7067 - disc_loss: 0.0954 - rmse: 0.0522 - val_loss: 0.0914
Epoch 1793/2000
1/1 [========

1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6979 - disc_loss: 0.0896 - rmse: 0.0747 - val_loss: 0.0574
Epoch 1845/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.5989 - disc_loss: 0.0924 - rmse: 0.0507 - val_loss: 0.0930
Epoch 1846/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6757 - disc_loss: 0.0960 - rmse: 0.0735 - val_loss: 0.0704
Epoch 1847/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6394 - disc_loss: 0.0932 - rmse: 0.0626 - val_loss: 0.0914
Epoch 1848/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.5973 - disc_loss: 0.0882 - rmse: 0.0556 - val_loss: 0.0641
Epoch 1849/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6333 - disc_loss: 0.0931 - rmse: 0.0627 - val_loss: 0.0658
Epoch 1850/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6734 - disc_loss: 0.0887 - rmse: 0.0641 - val_loss: 0.0589
Epoch 1851/2000
1/1 [========

1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6794 - disc_loss: 0.0915 - rmse: 0.1099 - val_loss: 0.0984
Epoch 1903/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7166 - disc_loss: 0.0937 - rmse: 0.1057 - val_loss: 0.1049
Epoch 1904/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.5820 - disc_loss: 0.0857 - rmse: 0.0693 - val_loss: 0.0688
Epoch 1905/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7301 - disc_loss: 0.0880 - rmse: 0.0993 - val_loss: 0.0690
Epoch 1906/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6479 - disc_loss: 0.0921 - rmse: 0.0753 - val_loss: 0.0807
Epoch 1907/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.6532 - disc_loss: 0.0996 - rmse: 0.0794 - val_loss: 0.1282
Epoch 1908/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6862 - disc_loss: 0.0877 - rmse: 0.0885 - val_loss: 0.1789
Epoch 1909/2000
1/1 [========

1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7627 - disc_loss: 0.0958 - rmse: 0.0771 - val_loss: 0.1586
Epoch 1961/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 0.8772 - disc_loss: 0.0937 - rmse: 0.1277 - val_loss: 0.1081
Epoch 1962/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7679 - disc_loss: 0.0798 - rmse: 0.0956 - val_loss: 0.0903
Epoch 1963/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7725 - disc_loss: 0.0884 - rmse: 0.0609 - val_loss: 0.1215
Epoch 1964/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.9276 - disc_loss: 0.0880 - rmse: 0.1253 - val_loss: 0.0816
Epoch 1965/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.7563 - disc_loss: 0.0890 - rmse: 0.1206 - val_loss: 0.1052
Epoch 1966/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 0.6446 - disc_loss: 0.0848 - rmse: 0.0723 - val_loss: 0.1046
Epoch 1967/2000
1/1 [========

row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
interpol flag :  [True, True]
folder :  data/han/인구/
colum_idx :  :,5:
file_names[idx] :  [['1_2016.xlsx', '1_2017.xlsx', '1_2018.xlsx'], ['2_2016.xlsx', '2_2017.xlsx', '2_2018.xlsx'], ['3_2016.xlsx', '3_2017.xlsx', '3_2018.xlsx'], ['4_2016.xlsx', '4_2017.xlsx', '4_2018.xlsx'], ['5_2016.xlsx', '5_2017.xlsx', '5_2018.xlsx'], ['6_2016.xlsx', '6_2017.xlsx', '6_2018.xlsx'], ['7_2016.xlsx', '7_2017.xlsx', '7_2018.xlsx'], ['8_2016.xlsx', '8_2017.xlsx', '8_2018.xlsx'], ['9_2016.xlsx', '9_2017.xlsx', '9_2018.xlsx'], ['10_2016.xlsx', '10_2017.xlsx', '10_2018.xlsx'], ['11_2016.xlsx', '11_2017.xlsx', '11_2018.xlsx'], ['12_2016.xlsx', '12_2017.xlsx', '12_2018.xlsx']]
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in
row in


In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [16]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  2
out_num_features :  1


In [17]:
#ctrl f -- 타겟

print(real_df_all.shape)
rnn_target_column = 'do'
print(rnn_target_column)



(26304, 1587)
do


In [18]:

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

train_df, test_df, val_df = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, val=1, test=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (18412, 1587) val_df.shape :  (5261, 1587) test_df.shape: (2631, 1587)


In [19]:
#fake_df.shape[0]/8760

In [20]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [21]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

In [22]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/han/models/han_pull/do/


In [23]:
rnn_epochs, rnn_in_setps, __RNN_TRAINING__

(15, 240, True)

## 모델 학습

In [24]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=2, steps_per_epoch = 2,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    #training_flag=True, checkpoint_path=model_path+"gru.ckpt", continue_train=False)

Epoch 1/15
5/5 [==============================] - 191s 38s/step - loss: 0.8158 - mean_absolute_error: 0.7174 - nse: -0.0722 - val_loss: 0.3371 - val_mean_absolute_error: 0.4481 - val_nse: 0.2589
Epoch 2/15
5/5 [==============================] - 175s 35s/step - loss: 0.4311 - mean_absolute_error: 0.5165 - nse: 0.4071 - val_loss: 0.2998 - val_mean_absolute_error: 0.4203 - val_nse: 0.3422
Epoch 3/15
5/5 [==============================] - 174s 35s/step - loss: 0.3196 - mean_absolute_error: 0.4415 - nse: 0.5859 - val_loss: 0.2807 - val_mean_absolute_error: 0.4049 - val_nse: 0.3819
Epoch 4/15
5/5 [==============================] - 174s 35s/step - loss: 0.2439 - mean_absolute_error: 0.3858 - nse: 0.6829 - val_loss: 0.2823 - val_mean_absolute_error: 0.4050 - val_nse: 0.3805
Epoch 5/15
5/5 [==============================] - 175s 35s/step - loss: 0.2046 - mean_absolute_error: 0.3477 - nse: 0.7084 - val_loss: 0.2863 - val_mean_absolute_error: 0.4070 - val_nse: 0.3724
Epoch 6/15
5/5 [=============

In [31]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)
    
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",real_df_all.shape[0])
print(teg_check)

print('all_nse : ', val_nse['GRU'])
print('all_pbias : ', val_pbias['GRU'])
print('all_mae : ', _mae)
print('all_rmse : ', _rmse)
print('all_Rs : ', _Rs)
print('all_R : ', _R)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

save/han/models/han_pull/do/
year : 2016 ~ 2018
run :  range(0, 16)
target :  do   2
length :  26304
train=7, val=1, test=2
all_nse :  [0.84637691]
all_pbias :  [-0.24495366]
all_mae :  0.47376448714737907
all_rmse :  0.6511327965740017
all_Rs :  0.8642122850815257
all_R :  0.9296301872688546


In [32]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('train_nse : ', val_nse['GRU'])
print('train_pbias : ', val_pbias['GRU'])
print('train_mae : ', _mae)
print('train_rmse : ', _rmse)
print('train_Rs : ', _Rs)
print('train_R : ', _R)

#print('nse_train : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
train_nse :  [0.8756569]
train_pbias :  [-0.55699015]
train_mae :  0.42141071139225733
train_rmse :  0.5814063142768259
train_Rs :  0.9001360079792404
train_R :  0.9487549778416134


In [33]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('val_nse : ', val_nse['GRU'])
print('val_pbias : ', val_pbias['GRU'])
print('val_mae : ', _mae)
print('val_rmse : ', _rmse)
print('val_Rs : ', _Rs)
print('val_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
val_nse :  [0.82126024]
val_pbias :  [0.93532234]
val_mae :  0.4246462453727697
val_rmse :  0.5090651794809872
val_Rs :  0.8879389516561115
val_R :  0.942305126620943


In [34]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)

print('----------------------------')
print('test_nse : ', val_nse['GRU'])
print('test_pbias : ', val_pbias['GRU'])
print('test_mae : ', _mae)
print('test_rmse : ', _rmse)
print('test_Rs : ', _Rs)
print('test_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
test_nse :  [0.5408078]
test_pbias :  [0.77649425]
test_mae :  0.7019183732436802
test_rmse :  0.9243953641639204
test_Rs :  0.5450995604277826
test_R :  0.7383085807626664


In [29]:
#train_df.shape, val_df.shape, test_df.shape

In [30]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:

sub_flag = True
history_nse = []

bob = 0
b = 1

while sub_flag:
    

    
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
        training_flag=True, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    
    #nse, _, _, _ = compa(
        #model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        #predict_day = 4)
    
    nse, _, _, _, _, _, _, _ = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -4):
        b = 1
    if(bob > 0):
        b = -1

